# ETL Project:
## Building a SQL Database for Comparison and Modeling of Cryptocurrency and Stock Markets

### Team Members: Ashley Zhang, Steven Brown, Wiliam Enriquez, Isaac Medina

#### Purpose: 
The goal of this project was to build a database that contained the necessary information to make comparisons between stock market trends and cryptocurrency trade markets, with the possibility of using the same information for buidling predictive models in the future using machine learning/neural networks. 
#### Data Sources
<ul> 
    <li> https://www.cryptodatadownload.com/data/coinbase/ - For CSVs containing Historical Daily Market Prices and Metrics of Cryptocurrencys 
    <li> https://www.wsj.com/market-data - For CSVs containing Historical Daily Market Prices and Metrics of stock indices
        

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# Loading CSV FILES containing Market Data

csv1 = "Resource/Coinbase_BCHUSD_d.csv"
csv2 = "Resource/Coinbase_BTCUSD_d.csv"
csv3 = "Resource/Coinbase_EOSUSD_d.csv"
csv4 = "Resource/Coinbase_ETCUSD_d.csv"
csv5 = "Resource/Coinbase_ETHUSD_d.csv"
csv6 = "Resource/Coinbase_LINKUSD_d.csv"
csv7 = "Resource/Coinbase_LTCUSD_d.csv"
csv8 = "Resource/Coinbase_REPUSD_d.csv"
csv9 = "Resource/Coinbase_XLMUSD_d.csv"
csv10 = "Resource/Coinbase_XRPUSD_d.csv"
csv11 = "Resource/COMP HistoricalPrices.csv"
csv12 = "Resource/DJIA HistoricalPrices.csv"
csv13 = "Resource/RUT 2000 HistoricalPrices.csv"
csv14 = "Resource/SPX 500 HistoricalPrices.csv"

csv_list = [csv1, csv2, csv3, csv4, csv5, csv6, csv7, csv8, csv9, csv10, csv11, csv12, csv13, csv14]
names = ['bch', 'btc', 'eos', 'etc', 'eth', 'lnk', 'ltc', 'rep', 'xlm', 'xrp', 'NAS', 'DOW', 'RUT', 'SPX']

for index, csv in enumerate(csv_list):  # loop over files

    exec(f"{names[index]} = pd.read_csv(csv)")
    
## Note: all cryptocurrency names in lowercase letters, Market indices have capitalized 3 letter code

### Initial Look at our Data

In [3]:
bch.head() #bitcoin dollar df

,unix,date,symbol,open,high,low,close,Volume BCH,Volume USD
0,1599609600,9/9/2020,BCH/USD,222.28,224.10,218.53,222.64,840773.59,3806.41
1,1599523200,9/8/2020,BCH/USD,228.78,230.47,215.90,222.28,5037488.62,22526.33
2,1599436800,9/7/2020,BCH/USD,227.69,229.88,215.78,228.78,6226370.73,27795.53
3,1599350400,9/6/2020,BCH/USD,225.74,233.26,219.20,227.69,5232916.02,23027.00
4,1599264000,9/5/2020,BCH/USD,230.93,234.96,214.50,225.74,13992453.13,62468.00


In [3]:
NAS.head() #nasdaq composite score df 
#no symbol column in stock indices

,Date,Open,High,Low,Close,Symbol
0,11/30/2020,12224.25,12244.65,12027.16,12198.74,COMP
1,11/27/2020,12159.18,12236.23,12154.57,12205.85,COMP
2,11/25/2020,12053.89,12114.77,12020.95,12094.40,COMP
3,11/24/2020,11939.33,12049.88,11863.45,12036.79,COMP
4,11/23/2020,11916.76,11949.33,11796.53,11880.63,COMP


In [4]:
#ADD a symbol column to our stock index data frames as a marker to merge databases later
NAS['Symbol']='COMP'
DOW['Symbol']='DJIA'
RUT['Symbol']='RUT 2000'
SPX['Symbol']='SPX 500'

In [5]:
#Check to see the range of dates in our data for each file.. to see how far back each data file goes
min_dates = []
max_dates = []

for csv in csv_list:
    data = pd.read_csv(csv)
    data.columns = [x.lower() for x in data.columns]
    min_date = data['date'].min()
    max_date = data['date'].max()
    min_dates.append(min_date)
    max_dates.append(max_date)

In [6]:
data_dict = {'File': csv_list, 'Earliest Date': min_dates, 'Last Date': max_dates}
df = pd.DataFrame(data_dict)

In [7]:
#Convert Date Column into datetime type for proper sorting 
df['Earliest Date'] = pd.to_datetime(df['Earliest Date'])

In [8]:
df.sort_values(by='Earliest Date')
#This table shows the range of dates available for each cryptocurrency and stock index
#All stock index data goes back as far as 01/10/2014 
#btc (Bitcoin) data goes as far back as 01/01/2015
#xrp (Ripple) is the shortest df going back only to 01/01/2020
#Other cryptocurrencies vary in how far back they go 

,File,Earliest Date,Last Date
10,Resource/COMP HistoricalPrices.csv,2014-01-10,9/9/2020
11,Resource/DJIA HistoricalPrices.csv,2014-01-10,9/9/2020
12,Resource/RUT 2000 HistoricalPrices.csv,2014-01-10,9/9/2020
13,Resource/SPX 500 HistoricalPrices.csv,2014-01-10,9/9/2020
1,Resource/Coinbase_BTCUSD_d.csv,2015-01-01,9/9/2020
4,Resource/Coinbase_ETHUSD_d.csv,2017-01-01,9/9/2020
6,Resource/Coinbase_LTCUSD_d.csv,2017-01-01,9/9/2020
0,Resource/Coinbase_BCHUSD_d.csv,2018-01-01,9/9/2020
3,Resource/Coinbase_ETCUSD_d.csv,2019-01-01,9/9/2020
2,Resource/Coinbase_EOSUSD_d.csv,2020-01-01,9/9/2020


In [9]:
# #drop columns
# # NAS = NAS.drop(['Unnamed: 0'], axis=1)
# DOW = DOW.drop(['Unnamed: 0'], axis=1)
# RUT = RUT.drop(['Unnamed: 0'], axis=1)
# SPX = SPX.drop(['Unnamed: 0'], axis=1)


In [10]:
print(NAS.head())
print(DOW.head())
print(RUT.head())
print(SPX.head())

         Date      Open      High       Low     Close Symbol
0  11/30/2020  12224.25  12244.65  12027.16  12198.74   COMP
1  11/27/2020  12159.18  12236.23  12154.57  12205.85   COMP
2  11/25/2020  12053.89  12114.77  12020.95  12094.40   COMP
3  11/24/2020  11939.33  12049.88  11863.45  12036.79   COMP
4  11/23/2020  11916.76  11949.33  11796.53  11880.63   COMP
         Date      Open      High       Low     Close Symbol
0  11/30/2020  29854.51  29854.51  29463.64  29638.64   DJIA
1  11/27/2020  29911.33  30015.13  29819.98  29910.37   DJIA
2  11/25/2020  30058.87  30058.87  29806.25  29872.47   DJIA
3  11/24/2020  29746.11  30116.51  29746.11  30046.24   DJIA
4  11/23/2020  29332.82  29667.75  29332.82  29591.27   DJIA
         Date     Open     High      Low    Close    Symbol
0  11/30/2020  1854.87  1854.87  1813.56  1819.82  RUT 2000
1  11/27/2020  1846.26  1855.27  1845.50  1855.27  RUT 2000
2  11/25/2020  1852.62  1852.62  1831.95  1845.02  RUT 2000
3  11/24/2020  1819.52  1862

In [5]:
# #Add index column
list_dfs = [bch, btc, eos, etc, eth, lnk, ltc, rep, xlm, xrp, NAS, DOW, RUT, SPX]
for df in list_dfs:
    df['Index'] = np.arange(len(df))


In [6]:
NAS.head()

,Date,Open,High,Low,Close,Symbol,Index
0,11/30/2020,12224.25,12244.65,12027.16,12198.74,COMP,0
1,11/27/2020,12159.18,12236.23,12154.57,12205.85,COMP,1
2,11/25/2020,12053.89,12114.77,12020.95,12094.40,COMP,2
3,11/24/2020,11939.33,12049.88,11863.45,12036.79,COMP,3
4,11/23/2020,11916.76,11949.33,11796.53,11880.63,COMP,4


In [10]:
# #write stock index data back to csv files (includes stock symbol column)
NAS.to_csv('Resource/COMP HistoricalPrices.csv', index=False)
DOW.to_csv('Resource/DJIA HistoricalPrices.csv', index=False)
RUT.to_csv('Resource/RUT 2000 HistoricalPrices.csv', index=False)
SPX.to_csv('Resource/SPX 500 HistoricalPrices.csv', index=False)

bch.to_csv('Resource/Coinbase_BCHUSD_d.csv', index=False)
btc.to_csv('Resource/Coinbase_BTCUSD_d.csv', index=False)
eos.to_csv('Resource/Coinbase_EOSUSD_d.csv', index=False)
etc.to_csv('Resource/Coinbase_ETCUSD_d.csv', index=False)
eth.to_csv('Resource/Coinbase_ETHUSD_d.csv', index=False)
lnk.to_csv('Resource/Coinbase_LINKUSD_d.csv', index=False)
ltc.to_csv('Resource/Coinbase_LTCUSD_d.csv', index=False)
rep.to_csv('Resource/Coinbase_REPUSD_d.csv', index=False)
xlm.to_csv('Resource/Coinbase_XLMUSD_d.csv', index=False)
xrp.to_csv('Resource/Coinbase_XRPUSD_d.csv', index=False)